- https://github.com/google-gemini/gemini-fullstack-langgraph-quickstart
    - https://github.com/google-gemini/gemini-fullstack-langgraph-quickstart/blob/main/backend/src/agent/prompts.py
- 带着问题读源码
    - deep research 是如何实现的；
        - agentic logic
    - 前后端全栈设计；
    - 多模型的经济且有效的组合；

### full stack

- frontend/ ：包含使用 Vite 构建的 React 应用程序。
    - `http://localhost:5173/`
    - 技术选型
        - React: 一个由 Facebook 开发的库，用于构建用户界面。它的核心思想是把界面拆分成一个个独立、可复用的“组件”（Components）。
        - Vite: 一个现代化的前端构建工具和开发服务器。它的特点是“快”，能让你在修改代码后立刻看到浏览器中的变化。
        - TypeScript: JavaScript 的一个超集，增加了静态类型检查。如果你熟悉 Python 的类型提示（Type Hinting），可以把它理解为前端的类型提示，能帮助我们更早地发现错误。
    - `App.tsx`

```typescript
let initial_search_query_count = 0;
let max_research_loops = 0;
switch (effort) {
  case "low":
    initial_search_query_count = 1;
    max_research_loops = 1;
    break;
  case "medium":
    initial_search_query_count = 3;
    max_research_loops = 3;
    break;
  case "high":
    initial_search_query_count = 5;
    max_research_loops = 10;
    break;
}

...


thread.submit({
   messages: newMessages,
   initial_search_query_count: initial_search_query_count,
   max_research_loops: max_research_loops,
   reasoning_model: model,
});
```

- backend/ ：包含 LangGraph/FastAPI 应用程序，包括 research agent logic。
    - langgraph：state machine
- 启动 `make dev`（`Makefile` recipes）

```mermaid
sequenceDiagram
    participant User as 用户
    participant Browser as 浏览器 (前端 @ 5173)
    participant Backend as 后端服务 (Agent @ 2024)

    User->>+Browser: 1. 打开 http://localhost:5173
    Browser->>+Backend: 2. useStream hook 建立连接到 http://localhost:2024
    User->>Browser: 3. 输入问题并提交
    Browser->>Backend: 4. 发送包含问题的 API 请求 (thread.submit)
    activate Backend
    Note over Backend: Agent 开始处理...<br/>(生成查询, 网络搜索, 反思)
    Backend-->>Browser: 5. [流式] 返回处理过程中的事件
    deactivate Backend
    Browser->>User: 6. 实时更新UI，展示Agent活动
    activate Backend
    Backend-->>Browser: 7. [流式] 返回最终答案
    deactivate Backend
    Browser->>User: 8. 展示最终答案
```

### 模型组合

- 前端表单
    ```python
    const [effort, setEffort] = useState("medium");
    const [model, setModel] = useState("gemini-2.5-flash-preview-04-17");
    ```
    - gemini-2.0-flash
    - gemini-2.5-flash-preview-04-17
    - gemini-2.5-pro-preview-05-06
- 后端
    - query_generator_model：gemini-2.0-flash
    - reflection_model：gemini-2.5-flash-preview-04-17
    - answer_model：gemini-2.5-pro-preview-05-06

### query_writer_instructions

- user (complex) query => single sub-queries
- format output
    ```python
    - Format your response as a JSON object with ALL three of these exact keys:
       - "rationale": Brief explanation of why these queries are relevant
       - "query": A list of search queries
    ```
- ice (in-context eamples)
```
Topic: What revenue grew more last year apple stock or the number of people buying an iphone
```json
{{
    "rationale": "To answer this comparative growth question accurately, we need specific data points on Apple's stock performance and iPhone sales metrics. These queries target the precise financial information needed: company revenue trends, product-specific unit sales figures, and stock price movement over the same fiscal period for direct comparison.",
    "query": ["Apple total revenue growth fiscal year 2024", "iPhone unit sales growth fiscal year 2024", "Apple stock price growth fiscal year 2024"],
}}
```
```

### langgraph

### tools

```python
response = genai_client.models.generate_content(
    model=configurable.query_generator_model,
    contents=formatted_prompt,
    config={
        "tools": [{"google_search": {}}],
        "temperature": 0,
    },
)
```

### citation